# Context-Aware AAC Testbed: LLM Experimentation

**Objective:** To determine if injecting specific contextual data (User Profile, Time, Location) into a Large Language Model (LLM) improves the accuracy and utility of phrase predictions for a user with Motor Neurone Disease (MND).

## The Hypothesis

Standard predictive text and generic LLMs fail AAC users because they prioritize "polite conversation" over "functional tools." We hypothesize that by layering **Static Context** (User Profile) and **Dynamic Context** (Time/Location) over speech input, we can move from generic chat to precise intent prediction.

**Subject Persona:** "Dave" – Late-stage MND, telegraphic speech, developer background.
**Critical Constraints:** High fatigue (limited breath for speech), temperature dysregulation, dependence on specific equipment (NIV Mask, Fan).

## 1. Setup & Repository Cloning
We install the necessary libraries and **clone the GitHub repository** to access the context JSON files (`dave_context.json`, etc.) directly.

In [ ]:
!pip install llm llm-gemini pandas matplotlib seaborn

import os
import json
import llm
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from google.colab import userdata

# --- 1. CLONE REPOSITORY ---
# This pulls the JSON files from your GitHub so you don't have to upload them manually.
repo_url = "https://github.com/willwade/ContextAwareTestBed.git"
repo_name = "ContextAwareTestBed"

if not os.path.exists(repo_name):
    print(f"🔄 Cloning {repo_name}...")
    !git clone $repo_url
else:
    print(f"✅ {repo_name} already exists. Pulling latest changes...")
    %cd $repo_name
    !git pull
    %cd ..

# Change working directory into the repo so we can see the files
if os.path.exists(repo_name):
    os.chdir(repo_name)
    print(f"📂 Changed directory to: {os.getcwd()}")
    print("📄 Files available:")
    !ls

# --- 2. API KEY SETUP ---
try:
    os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')
except:
    print("\n⚠️ Colab Secret not found.")
    os.environ["GOOGLE_API_KEY"] = input("Enter Google API Key: ")

# Configure llm
model = llm.get_model("gemini-2.5-pro") 
print(f"\n✅ Setup Complete. Using model: {model.model_id}")

## 2. Load Data
Now that the repository is cloned, we can load the JSON files directly from the local file system.

In [ ]:
# Load the Static Profile (The Knowledge Graph)
try:
    with open('dave_context.json', 'r') as f:
        dave_profile = json.load(f)
    print("✅ Dave's Profile Loaded.")
except FileNotFoundError:
    print("❌ ERROR: 'dave_context.json' not found. Did the git clone work?")

# Load the Transcripts (The Scenarios)
try:
    with open('transcript_data_2.json', 'r') as f:
        transcript_strict = json.load(f)
    with open('transcript_vague.json', 'r') as f:
        transcript_vague = json.load(f)
    print("✅ Transcripts Loaded.")
except FileNotFoundError:
    print("❌ ERROR: Transcript files not found.")

## 3. Experiment 1: The Baseline (Context vs. Noise)

**Hypothesis:** Providing environmental data (Time, Location, People) without speech will result in hallucinations, while Speech + Profile will provide a strong baseline.

We compare 5 hypotheses:
* **H1:** Time Only
* **H2:** Time + Who
* **H3:** Time + Who + Location
* **H5:** Speech + Profile (Baseline)
* **H4:** Full Context (Speech + Profile + Time + Who + Location)

In [ ]:
# --- CONFIGURATION ---
MODEL_NAME = "gemini-2.5-pro" 

def generate_prediction(model, system_prompt, user_prompt):
    try:
        # Low temp for consistency
        response = model.prompt(user_prompt, system=system_prompt, temperature=0.2)
        text = response.text().strip()
        return text.replace("\n", " ")
    except Exception as e:
        return f"Error: {e}"

def evaluate_intent_match(model, target, prediction):
    """
    Judges the semantic closeness of the prediction to the target (1-10).
    """
    judge_system = "You are a semantic evaluator for an AAC system."
    judge_prompt = f"""
    Compare these two phrases.
    1. TARGET INTENT: "{target}"
    2. AI PREDICTION: "{prediction}"
    
    Rate the similarity of the INTENT (Actionability/Meaning) on a scale of 1 to 10.
    1 = Completely wrong/harmful.
    5 = Vague or related topic but wrong action.
    10 = Perfect match.
    
    Return ONLY the integer.
    """
    try:
        response = model.prompt(judge_prompt, system=judge_system, temperature=0.0)
        score = "".join(filter(str.isdigit, response.text()))
        return int(score) if score else 0
    except:
        return 0

def run_strict_experiment():
    if 'dave_profile' not in globals() or 'transcript_strict' not in globals():
        print("❌ Data not loaded. Cannot run experiment.")
        return pd.DataFrame()

    model = llm.get_model(MODEL_NAME)
    
    base_system_prompt = f"""
    You are a Predictive AAC System for a user named Dave.
    USER PROFILE:
    {json.dumps(dave_profile, indent=2)}
    INSTRUCTIONS:
    1. Predict the most likely short phrase Dave wants to say.
    2. Base prediction ONLY on the INPUT DATA provided.
    3. Output ONLY the predicted phrase.
    """

    print(f"Running Full Spectrum Experiment...")
    print("=" * 70)

    results = []

    for turn in transcript_strict:
        print(f"Processing ID {turn['id']} ({turn['target_ground_truth']})...")
        
        # Data Points
        time = turn["metadata"]["time"]
        participants = ", ".join(turn["metadata"]["active_participants"])
        location = turn["metadata"]["location"]
        prev_utterance = f"Previous Speaker said: '{turn['dialogue_history']['last_utterance']}'"
        
        # --- H1: TIME ONLY ---
        h1_pred = generate_prediction(model, base_system_prompt, f"INPUT: Time: {time}")
        h1_score = evaluate_intent_match(model, turn["target_ground_truth"], h1_pred)

        # --- H2: TIME + WHO ---
        h2_pred = generate_prediction(model, base_system_prompt, f"INPUT: Time: {time}. People: {participants}")
        h2_score = evaluate_intent_match(model, turn["target_ground_truth"], h2_pred)

        # --- H3: TIME + WHO + LOC ---
        h3_pred = generate_prediction(model, base_system_prompt, f"INPUT: Time: {time}. People: {participants}. Location: {location}")
        h3_score = evaluate_intent_match(model, turn["target_ground_truth"], h3_pred)

        # --- H5: SPEECH ONLY (No environmental context) ---
        h5_pred = generate_prediction(model, base_system_prompt, f"INPUT: {prev_utterance}")
        h5_score = evaluate_intent_match(model, turn["target_ground_truth"], h5_pred)

        # --- H4: FULL CONTEXT (All inputs) ---
        h4_pred = generate_prediction(model, base_system_prompt, f"INPUT: Time: {time}. People: {participants}. Location: {location}. {prev_utterance}")
        h4_score = evaluate_intent_match(model, turn["target_ground_truth"], h4_pred)

        results.append({
            'ID': turn['id'],
            'Target': turn['target_ground_truth'],
            'H1_Time': h1_pred, 'H1_Score': h1_score,
            'H2_Who': h2_pred, 'H2_Score': h2_score,
            'H3_Loc': h3_pred, 'H3_Score': h3_score,
            'H5_Speech': h5_pred, 'H5_Score': h5_score,
            'H4_Full': h4_pred, 'H4_Score': h4_score
        })

    return pd.DataFrame(results)

df_results = run_strict_experiment()
df_results

## 4. Visualizing the Context Advantage
We compare **H5 (Speech + Profile)** vs **H4 (Full Context)** to see where context closes the gap.

In [ ]:
if not df_results.empty:
    # Setup the plot
    plt.figure(figsize=(12, 6))
    sns.set_style("whitegrid")

    # Melt dataframe for Seaborn
    df_melted = df_results.melt(id_vars=['ID', 'Target'], 
                                value_vars=['H5_Score', 'H4_Score'], 
                                var_name='Hypothesis', 
                                value_name='Accuracy_Score')

    # Custom colors: Grey for Speech Only, Green for Context
    palette = {"H5_Score": "#95a5a6", "H4_Score": "#2ecc71"}

    ax = sns.barplot(x='ID', y='Accuracy_Score', hue='Hypothesis', data=df_melted, palette=palette)

    plt.title('Impact of Context Awareness on AAC Prediction Accuracy', fontsize=16)
    plt.xlabel('Scenario ID', fontsize=12)
    plt.ylabel('Semantic Accuracy Score (1-10)', fontsize=12)
    plt.ylim(0, 11)
    plt.legend(title='Model Configuration', labels=['H5: Speech Only', 'H4: Full Context'])

    # Annotate the bars with the target intent
    for i in range(len(df_results)):
        row = df_results.iloc[i]
        plt.text(i, 10.2, row['Target'], ha='center', fontsize=9, rotation=0, color='#34495e')

    plt.tight_layout()
    plt.show()
else:
    print("No results to plot.")

## 5. Experiment 2: The Ablation Test (Value of Profile)

**Hypothesis:** If we strip away the User Profile (The Knowledge Graph), the utility of the system will collapse.

**Scientific Control Update:** To ensure a fair test, both the "Smart" and "Generic" models now use the **exact same system prompt instructions**. The only difference is that the "Smart" model receives the JSON Profile, while the "Generic" model receives an empty placeholder. This isolates the profile data as the single independent variable.

In [ ]:
def run_ablation_experiment():
    if 'dave_profile' not in globals() or 'transcript_vague' not in globals():
        print("❌ Data not loaded. Cannot run experiment.")
        return

    model = llm.get_model(MODEL_NAME)

    # BASE PROMPT TEMPLATE
    # We use the same template for both to ensure a fair scientific control.
    prompt_template = """
    You are an AAC (Augmentative and Alternative Communication) assistant for a user named Dave.
    Your goal is to predict the short phrase Dave wants to say based on the context.

    USER PROFILE:
    {profile_data}

    INSTRUCTIONS:
    Predict his response based on the input speech. Short phrases only.
    """

    print(f"Running 'Speech Only' Ablation Test on {MODEL_NAME}")
    print("=" * 60)
    print(f"{'INPUT SPEECH':<40} | {'SMART (With Profile)':<25} | {'GENERIC (No Profile)'}")
    print("-" * 80)

    for turn in transcript_vague:
        user_input = f"Previous speaker said: '{turn['last_utterance']}'"

        # 1. Run Smart (Inject Profile)
        smart_system = prompt_template.format(profile_data=json.dumps(dave_profile))
        smart_pred = model.prompt(user_input, system=smart_system, temperature=0.1).text().strip()
        
        # 2. Run Generic (Inject Empty Profile)
        dumb_system = prompt_template.format(profile_data="[NO PROFILE DATA AVAILABLE]")
        raw_pred = model.prompt(user_input, system=dumb_system, temperature=0.1).text().strip()
        
        # Output
        print(f"'{turn['last_utterance'][:35]:<38}' | {smart_pred:<25} | {raw_pred}")

run_ablation_experiment()

## 6. Experiment 3: The Synthesis (Value of Time)

**Hypothesis:** Time is the "Key" that unlocks ambiguous speech variables (e.g., "The Usual").

In [ ]:
def run_synthesis_experiment():
    if 'dave_profile' not in globals():
        print("❌ Data not loaded. Cannot run experiment.")
        return

    model = llm.get_model(MODEL_NAME)

    scenarios = [
        {
            "speech": "Do you want the usual?",
            "time": "08:00", 
            "context_note": "Morning"
        },
        {
            "speech": "Do you want the usual?",
            "time": "20:00", 
            "context_note": "Evening"
        }
    ]

    print(f"Running Temporal Context Test on {MODEL_NAME}")
    print("=" * 60)

    for sc in scenarios:
        current_system = f"""
        You are an AAC assistant for Dave.
        
        USER PROFILE:
        {json.dumps(dave_profile, indent=2)}
        
        CURRENT CONTEXT:
        Time: {sc['time']}
        
        INSTRUCTIONS:
        Predict Dave's response based on the input speech. 
        If the speech is vague (e.g. "the usual"), use the TIME and the PROFILE to guess the routine.
        Output only the short phrase response.
        """
        
        input_text = f"Kelsey said: '{sc['speech']}'"
        
        # Run prediction
        prediction = model.prompt(input_text, system=current_system, temperature=0.1).text().strip()
        
        print(f"Time: {sc['time']} ({sc['context_note']})")
        print(f"Input: '{sc['speech']}'")
        print(f"Prediction: {prediction}")
        print("-" * 40)

run_synthesis_experiment()

## 7. New Experiment 4: The "Input Compression" Test

Recent research (e.g., *"Using Large Language Models to Accelerate Communication for Users with Severe Motor Impairments"*, Cai et al. 2023) suggests that users benefit most from **extreme abbreviation** to save motor effort.

**Hypothesis:** A Profile-Aware model can correctly expand ambiguous initialisms (e.g., "f o") into specific needs (e.g., "Fan on"), whereas a Generic model will fail.

This test compares: 
* **Generic Model:** Expanding abbreviations without context.
* **Context Model:** Expanding abbreviations WITH Dave's profile.

In [ ]:
def run_abbreviation_test():
    model = llm.get_model(MODEL_NAME)

    # --- TEST CASES ---
    abbreviations = [
        {"input": "f o", "target": "Fan on"},
        {"input": "w s", "target": "Window shut"},
        {"input": "n b a", "target": "Neck brace adjust"},
        {"input": "m o", "target": "Mask on"}
    ]

    # --- SYSTEM PROMPTS ---
    
    # 1. DAVE-AWARE (Context)
    smart_system = f"""
    You are an intelligent AAC abbreviation expander for Dave.
    USER PROFILE:
    {json.dumps(dave_profile, indent=2)}
    
    INSTRUCTIONS:
    Expand the user's initialism (e.g. 'f o') into the most likely full phrase based on his profile and common requests.
    Output ONLY the expansion.
    """

    # 2. GENERIC (No Context)
    dumb_system = """
    You are a helpful predictive text assistant.
    Expand the user's initialism (abbreviation) into the most likely full english phrase.
    Output ONLY the expansion.
    """

    print(f"Running Abbreviation Expansion Test...")
    print("=" * 80)
    print(f"{'INPUT':<10} | {'SMART (Profile)':<25} | {'GENERIC (No Profile)'}")
    print("-" * 80)

    for case in abbreviations:
        user_input = f"User typed: '{case['input']}'"

        # Run Smart
        smart_pred = model.prompt(user_input, system=smart_system, temperature=0.0).text().strip()
        
        # Run Generic
        generic_pred = model.prompt(user_input, system=dumb_system, temperature=0.0).text().strip()
        
        print(f"'{case['input']:<8}' | {smart_pred:<25} | {generic_pred}")

run_abbreviation_test()

## Conclusion & Research Validation

### 1. The "Deictic" and "Temporal" Gaps
Our original experiments proved that **Time** and **Static Profile** are the most critical disambiguators for vague speech.

### 2. Validation from Literature
New research supports our findings and suggests further improvements:

* **The Input Upgrade:** Cai et al. (Google, arXiv:2312.01532) showed that LLMs can reduce motor effort by **57%** using abbreviation expansion. Our Experiment 4 confirms that **Context is required** for this to work effectively. Without the profile, "f o" becomes "fuck off" or "for one" instead of "Fan on."
* **The Logic Upgrade:** Abrams & Scheutz (NAACL 2022) highlight that **Social Norms** guide reference resolution. This suggests our future Social Graph should not just list "Roles" (Nurse vs. Mom) but "Interaction Styles" (Formal vs. Intimate) to better predict the *tone* of the response.